# INPUTS

`date`: The day to scrape matches from (defaults to today)

`tournament`: The current ATP tournament

In [2]:
# User Inputs
date = "2022-07-10"
tournament = "Wimbledon"

# SCRAPER

## Import Packages

In [3]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime

## Scrape Upcoming Matches

Using the Tennis Live API, two separate API calls are made. The first retrieves a list of matches for a specified day and the second retrieves a list of current player rankings. Those are then combined to give us new rows to append to the bottom of the `full_df` in the `ATP_data_cleaning` notebook.

In [4]:
def scrapeFutureMatches(date): # date in format yyyy-mm-dd
    
    # Matches API request
    # Concatenate base URL and date
    base_url = "https://tennis-live-data.p.rapidapi.com/matches-by-date/"
    url = base_url + date

    # API call to get the matches for a given day
    headers = {
        "X-RapidAPI-Key": "00229723ddmsh7d1fdfd7b967bd6p1e6ca6jsn0b3e25e2c461",
        "X-RapidAPI-Host": "tennis-live-data.p.rapidapi.com"
    }
    matches_response = requests.request("GET", url, headers=headers)

    # flatten and format the JSON
    matches_df = pd.json_normalize(matches_response.json()['results'])
    matches_df = matches_df.explode('matches')
    matches_df = matches_df['matches'].apply(pd.Series)[['status', 'home_player', 'away_player', 'date']]
    matches_df['home_player'] = matches_df['home_player'].apply(lambda x: str(x.split()[0]))
    matches_df['away_player'] = matches_df['away_player'].apply(lambda x: str(x.split()[0]))

    # duplicate rows to make the dataframe tidy by player
    opp = matches_df.rename(columns={'home_player': 'away_player', 'away_player': 'home_player'})
    matches_df = pd.concat([matches_df, opp]).rename(columns={'home_player': 'player', 'away_player': 'opponent'}) 
    matches_df = matches_df.loc[0]

    

    # Rankings API request
    # API call to get the rank points for ATP players
    url = "https://tennis-live-data.p.rapidapi.com/rankings/ATP"
    headers = {
        "X-RapidAPI-Key": "00229723ddmsh7d1fdfd7b967bd6p1e6ca6jsn0b3e25e2c461",
        "X-RapidAPI-Host": "tennis-live-data.p.rapidapi.com"
    }
    rankings_response = requests.request("GET", url, headers=headers)

    # Flatten the JSON and rename columns
    rankings_df = pd.json_normalize(rankings_response.json()['results']['rankings'])
    rankings_df = rankings_df[['full_name', 'ranking_points']]
    rankings_df['player'] = rankings_df['full_name'].apply(lambda x: str(x.split()[-1]))
    rankings_df = rankings_df.rename(columns={'full_name': 'player_full_name', 'ranking_points': 'player_ranking_points'})
    
    
    
    # Joining the two dataframes
    # Merge to get player names and rank points
    matches_df['player']=matches_df['player'].astype(str)
    rankings_df['player']=rankings_df['player'].astype(str)
    info_df = matches_df.merge(rankings_df, on='player', how='left')

    # Merge to get opponent names and rank points
    opp_rankings_df = rankings_df.rename(columns={'player_full_name': 'opp_full_name', 'player_ranking_points': 'opp_ranking_points', 'player': 'opponent'})
    info_df = info_df.merge(opp_rankings_df, on='opponent', how='left')
    
    # Clean and save resulting dataframe
    matches = info_df.drop(columns = info_df[['player', 'opponent']])
    matches = matches.rename(columns={'player_full_name': 'name', 'opp_full_name': 'opp_name', 'player_ranking_points': 'rank_points', 'opp_ranking_points': 'opp_rank_points'})
    matches['tourney_date'] = matches['date'].apply(lambda x: x[:10].replace('-', ''))
    future_matches = matches.drop(['status', 'date'], axis=1)
    
    return future_matches

In [5]:
# Change input date to today
upcoming_matches_df = scrapeFutureMatches("2022-07-10")
upcoming_matches_df.head()

,name,rank_points,opp_name,opp_rank_points,tourney_date
0,Novak Djokovic,4770,Nick Kyrgios,710,20220710
1,Nick Kyrgios,710,Novak Djokovic,4770,20220710


## Scrape PrizePicks Lines

Use helper function from the [PrizePicks API documentation](https://github.com/PrizePicks-Analytics/PrizePicks-API) to return data.

In [6]:
# Use helper function
def call_endpoint(url, max_level=3, include_new_player_attributes=False):
    '''
    takes: 
        - url (str): the API endpoint to call
        - max_level (int): level of json normalizing to apply
        - include_player_attributes (bool): whether to include player object attributes in the returned dataframe
    returns:
        - df (pd.DataFrame): a dataframe of the call response content
    '''
    resp = requests.get(url).json()
    data = pd.json_normalize(resp['data'], max_level=max_level)
    included = pd.json_normalize(resp['included'], max_level=max_level)
    if include_new_player_attributes:
        inc_cop = included[included['type'] == 'new_player'].copy().dropna(axis=1)
        data = pd.merge(data,
                        inc_cop,
                        how='left',
                        left_on=['relationships.new_player.data.id', 'relationships.new_player.data.type'],
                        right_on=['id', 'type'],
                        suffixes=('', '_new_player'))
    return data

# Create dataframe using the helper function
url = 'https://partner-api.prizepicks.com/projections?per_page=100'
df = call_endpoint(url, include_new_player_attributes=True)
lines = df[['attributes.league', 'attributes.team', 'attributes.name', 'attributes.stat_type', 'attributes.line_score', 'attributes.start_time']]
lines = lines.rename(columns={
    "attributes.line_score": "prizepicks_line",
    "attributes.start_time": "Start Time",
    "attributes.stat_type": "Stat Type",
    "attributes.name": "name",
    "attributes.team": "Team Name",
    "attributes.league": "League"
})

# Format start time
lines['Start Date'] = ""
for i in range(len(lines['Start Time'])):
    temp_date = datetime.fromisoformat(lines['Start Time'][i])
    lines['Start Date'][i] = temp_date.date()
    lines['Start Time'][i] = temp_date.hour
lines['tourney_date'] = lines['Start Date'].apply(lambda x: x.strftime("%Y%m%d"))
    
    
# Filter to only include tennis, only fantasy points
lines = lines[lines['League'] == 'TENNIS']
lines = lines[lines['Stat Type'] == 'Fantasy Score']

# Drop unnecessary cols
prizepicks_df = lines.drop(columns=['League', 'Team Name', 'Stat Type', 'Start Time', 'Start Date'])

# Show dataframe and save to CSV
prizepicks_df.head()
# lines.to_csv(r'June 28 Tennis.csv')

,name,prizepicks_line,tourney_date
5,Mackenzie McDonald,15.5,20220808
7,Emil Ruusuvuori,20.0,20220808
8,Karen Khachanov,20.5,20220808
10,Jenson Brooksby,15.0,20220808
11,Taylor Fritz,20.5,20220808


## Combine Dataframes

`prizepicks_line` is not a column that exists in the historical ATP dataset, so it will only be used for the sake of assessing whether our predictions outperform expectations or not.

In [7]:
scraped_future_matches = prizepicks_df.merge(upcoming_matches_df, on=['name', 'tourney_date'])

# DATA CLEANING

## Load Data

Data is loaded from Jack Sackmann's tennis_atp GitHub repository by year and concatenated together.

In [8]:
# Load in ATP data 1997 to 2021
atp_1997 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_1997.csv", engine="python", encoding="utf8")
atp_1998 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_1998.csv", engine="python", encoding="utf8")
atp_1999 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_1999.csv", engine="python", encoding="utf8")
atp_2000 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2000.csv", engine="python", encoding="utf8")
atp_2001 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2001.csv", engine="python", encoding="utf8")
atp_2002 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2002.csv", engine="python", encoding="utf8")
atp_2003 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2003.csv", engine="python", encoding="utf8")
atp_2004 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2004.csv", engine="python", encoding="utf8")
atp_2005 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2005.csv", engine="python", encoding="utf8")
atp_2006 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2006.csv", engine="python", encoding="utf8")
atp_2007 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2007.csv", engine="python", encoding="utf8")
atp_2008 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2008.csv", engine="python", encoding="utf8")
atp_2009 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2009.csv", engine="python", encoding="utf8")
atp_2010 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2010.csv", engine="python", encoding="utf8")
atp_2011 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2011.csv", engine="python", encoding="utf8")
atp_2012 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2012.csv", engine="python", encoding="utf8")
atp_2013 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2013.csv", engine="python", encoding="utf8")
atp_2014 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2014.csv", engine="python", encoding="utf8")
atp_2015 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2015.csv", engine="python", encoding="utf8")
atp_2016 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2016.csv", engine="python", encoding="utf8")
atp_2017 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2017.csv", engine="python", encoding="utf8")
atp_2018 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2018.csv", engine="python", encoding="utf8")
atp_2019 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2019.csv", engine="python", encoding="utf8")
atp_2020 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2020.csv", engine="python", encoding="utf8")
atp_2021 = pd.read_csv("https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_matches_2021.csv", engine="python", encoding="utf8")

# Append these datasets together
atp = pd.concat([
    atp_1997,atp_1998,atp_1999,atp_2000,atp_2001,atp_2002,atp_2003,atp_2004,atp_2005,
    atp_2006,atp_2007,atp_2008,atp_2009,atp_2010,atp_2011,atp_2012,atp_2013,atp_2014,
    atp_2015,atp_2016,atp_2017,atp_2018,atp_2019,atp_2020,atp_2021
])

# Return the head of the dataframe
atp.head()

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,1997-339,Adelaide,Hard,32,A,19961230,1,102025,NaN,NaN,...,18.0,7.0,18.0,8.0,5.0,9.0,47.0,875.0,3.0,3564.0
1,1997-339,Adelaide,Hard,32,A,19961230,2,101647,NaN,NaN,...,25.0,21.0,9.0,9.0,1.0,3.0,44.0,886.0,73.0,636.0
2,1997-339,Adelaide,Hard,32,A,19961230,3,102158,NaN,NaN,...,36.0,25.0,13.0,9.0,5.0,7.0,62.0,741.0,202.0,194.0
3,1997-339,Adelaide,Hard,32,A,19961230,4,101772,NaN,WC,...,51.0,40.0,17.0,15.0,6.0,10.0,133.0,339.0,40.0,993.0
4,1997-339,Adelaide,Hard,32,A,19961230,5,101889,4.0,NaN,...,41.0,31.0,22.0,14.0,2.0,5.0,36.0,1039.0,71.0,661.0


## Data Cleaning

Data is cleaned by computing fantasy points (based on the PrizePicks scoring system), adding a `grand_slam` indicator variable, filtering out erroneous data, and removing unecessary columns.

In [9]:
# Data cleaning functions

# Function to calc fantasy points
def calcFantasyPoints(row):
    # Separate scores into winner's and loser's games won by set
    winner_score = row.score.replace('-', ' ').split()[::2]
    loser_score = row.score.replace('-', ' ').split()[1::2]
    
    # Convert to integer type and sum games won
    winner_games_won = sum(list(map(int, winner_score)))
    loser_games_won = sum(list(map(int, loser_score)))
    
    # Calculate sets won
    winner_sets_won = findSetsWon(winner_score, loser_score)
    loser_sets_won = findSetsWon(loser_score, winner_score)

    # Find and return fantasy points
    winner_fantasy_pts = 10 + winner_games_won - loser_games_won + (3 * winner_sets_won) - (3 * loser_sets_won) + (0.5 * row['w_ace']) - (0.5 * row['w_df'])
    loser_fantasy_pts = 10 + loser_games_won - winner_games_won + (3 * loser_sets_won) - (3 * winner_sets_won) + (0.5 * row['l_ace']) - (0.5 * row['l_df'])
    
    return pd.Series([winner_fantasy_pts, loser_fantasy_pts])

# Helper function to calculate sets won
def findSetsWon(player_sets, opponent_sets):
    sets = 0
    for i in range(len(player_sets)):
        if player_sets[i] > opponent_sets[i]:
            sets += 1
    return sets

In [10]:
# Remove matches that were not played to completion (and matches with scoring errors)
erroneous_strings = ['RET', 'Default', 'Apr', 'W/O', '[10]', 'DEF', 'Played', 'Unfinished', 'Walkover']
atp = atp[~atp.score.str.contains('|'.join(erroneous_strings))]

# Remove tiebreak games
atp['score'] = atp['score'].str.replace('\(.\)', '', regex=True)

# Adding columns
atp[['winner_fantasy_pts', 'loser_fantasy_pts']] = atp.apply(calcFantasyPoints, axis = 1)
atp['grand_slam'] = atp['tourney_name'].apply(lambda tourney: 1 if tourney in ['US Open', 'Wimbledon', 'Australian Open', 'Roland Garros'] else 0)

# Removing completely irrelevant columns
atp = atp.drop(columns = ['draw_size', 'tourney_level', 'match_num', 'winner_entry', 'loser_entry'])

atp.head()

,tourney_id,tourney_name,surface,tourney_date,winner_id,winner_seed,winner_name,winner_hand,winner_ht,winner_ioc,...,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,winner_fantasy_pts,loser_fantasy_pts,grand_slam
0,1997-339,Adelaide,Hard,19961230,102025,NaN,Mikael Tillstrom,R,185.0,SWE,...,8.0,5.0,9.0,47.0,875.0,3.0,3564.0,25.0,-4.5,0
1,1997-339,Adelaide,Hard,19961230,101647,NaN,Byron Black,R,175.0,ZIM,...,9.0,1.0,3.0,44.0,886.0,73.0,636.0,21.5,2.5,0
2,1997-339,Adelaide,Hard,19961230,102158,NaN,Patrick Rafter,R,185.0,AUS,...,9.0,5.0,7.0,62.0,741.0,202.0,194.0,22.0,-1.5,0
3,1997-339,Adelaide,Hard,19961230,101772,NaN,Andrei Cherkasov,R,180.0,RUS,...,15.0,6.0,10.0,133.0,339.0,40.0,993.0,15.0,5.0,0
4,1997-339,Adelaide,Hard,19961230,101889,4.0,Todd Woodbridge,R,178.0,AUS,...,14.0,2.0,5.0,36.0,1039.0,71.0,661.0,14.5,4.5,0


## Restructure Dataframe

Changes dataframe from winner-loser format to player-opponent format. Rows are also duplicated such that each player in a match has their own distinct row.

In [11]:
# create dataframe with winners as players, losers as opponents
winners_df = atp.rename(columns={
    'winner_fantasy_pts': 'fantasy_pts', 'loser_fantasy_pts': 'opp_fantasy_pts', # score data
    'winner_id': 'id', 'winner_seed': 'seed', 'winner_name': 'name', 'winner_hand': 'hand', 'winner_ht': 'ht', 'winner_ioc': 'ioc', 'winner_age': 'age', 'winner_rank': 'rank', 'winner_rank_points': 'rank_points', # player metadata
    'loser_id': 'opp_id', 'loser_seed': 'opp_seed', 'loser_entry': 'opp_entry', 'loser_name': 'opp_name', 'loser_hand': 'opp_hand', 'loser_ht': 'opp_ht', 'loser_ioc': 'opp_ioc', 'loser_age': 'opp_age', 'loser_rank': 'opp_rank', 'loser_rank_points': 'opp_rank_points', # opponent metadata
    'w_ace': 'ace', 'w_df': 'df', 'w_svpt': 'svpt', 'w_1stIn': 'FirstIn', 'w_1stWon': 'FirstWon', 'w_2ndWon': 'SecondWon', 'w_SvGms': 'SvGms', 'w_bpSaved': 'bpSaved', 'w_bpFaced' : 'bpFaced', # player stats
    'l_ace': 'opp_ace', 'l_df': 'opp_df', 'l_svpt': 'opp_svpt', 'l_1stIn': 'opp_FirstIn', 'l_1stWon': 'opp_FirstWon', 'l_2ndWon': 'opp_SecondWon', 'l_SvGms': 'opp_SvGms', 'l_bpSaved': 'opp_bpSaved', 'l_bpFaced': 'opp_bpFaced' # opponent stats 
})

# create dataframe with losers as players, winners as opponents
losers_df = atp.rename(columns={
    'loser_fantasy_pts': 'fantasy_pts', 'winner_fantasy_pts': 'opp_fantasy_pts', # score data
    'loser_id': 'id', 'loser_seed': 'seed', 'loser_name': 'name', 'loser_hand': 'hand', 'loser_ht': 'ht', 'loser_ioc': 'ioc', 'loser_age': 'age', 'loser_rank': 'rank', 'loser_rank_points': 'rank_points', # player metadata
    'winner_id': 'opp_id', 'winner_seed': 'opp_seed', 'winner_entry': 'opp_entry', 'winner_name': 'opp_name', 'winner_hand': 'opp_hand', 'winner_ht': 'opp_ht', 'winner_ioc': 'opp_ioc', 'winner_age': 'opp_age', 'winner_rank': 'opp_rank', 'winner_rank_points': 'opp_rank_points', # opponent metadata
    'l_ace': 'ace', 'l_df': 'df', 'l_svpt': 'svpt', 'l_1stIn': 'FirstIn', 'l_1stWon': 'FirstWon', 'l_2ndWon': 'SecondWon', 'l_SvGms': 'SvGms', 'l_bpSaved': 'bpSaved', 'l_bpFaced' : 'bpFaced', # player stats
    'w_ace': 'opp_ace', 'w_df': 'opp_df', 'w_svpt': 'opp_svpt', 'w_1stIn': 'opp_FirstIn', 'w_1stWon': 'opp_FirstWon', 'w_2ndWon': 'opp_SecondWon', 'w_SvGms': 'opp_SvGms', 'w_bpSaved': 'opp_bpSaved', 'w_bpFaced': 'opp_bpFaced' # opponent stats 
})

# join the two dataframes together
full_df = pd.concat([winners_df, losers_df]).reset_index()
full_df.head()

,index,tourney_id,tourney_name,surface,tourney_date,id,seed,name,hand,ht,...,opp_SvGms,opp_bpSaved,opp_bpFaced,rank,rank_points,opp_rank,opp_rank_points,fantasy_pts,opp_fantasy_pts,grand_slam
0,0,1997-339,Adelaide,Hard,19961230,102025,NaN,Mikael Tillstrom,R,185.0,...,8.0,5.0,9.0,47.0,875.0,3.0,3564.0,25.0,-4.5,0
1,1,1997-339,Adelaide,Hard,19961230,101647,NaN,Byron Black,R,175.0,...,9.0,1.0,3.0,44.0,886.0,73.0,636.0,21.5,2.5,0
2,2,1997-339,Adelaide,Hard,19961230,102158,NaN,Patrick Rafter,R,185.0,...,9.0,5.0,7.0,62.0,741.0,202.0,194.0,22.0,-1.5,0
3,3,1997-339,Adelaide,Hard,19961230,101772,NaN,Andrei Cherkasov,R,180.0,...,15.0,6.0,10.0,133.0,339.0,40.0,993.0,15.0,5.0,0
4,4,1997-339,Adelaide,Hard,19961230,101889,4.0,Todd Woodbridge,R,178.0,...,14.0,2.0,5.0,36.0,1039.0,71.0,661.0,14.5,4.5,0


## Append Future Matches

Reads in data for upcoming matches as a CSV from the `future_match_scraper` notebook. Requires user to set a value for the `tournament` and `date`.

In [13]:
# Read in future info file and add user inputs

scraped_future_matches = pd.read_csv("future_matches_test.csv") # REMOVE THIS LINE AFTER TESTING
scraped_future_matches['tourney_name'] = tournament

# Dataframe with info about all distinct players
unique_players = full_df.drop_duplicates(subset=['id'], keep='last')[['name', 'id', 'age', 'tourney_date', 'hand', 'ht']].rename(columns={'tourney_date': 'prev_tourney_date', 'age': 'prev_age'})
scraped_future_matches = scraped_future_matches.merge(unique_players, on='name')

# Helper function to calculate a player's age for the upcoming match
def calculateAge(tourney_date, prev_tourney_date, prev_age):
    # Add most recent age to the time between matches
    upcoming_date = datetime.strptime(str(tourney_date), "%Y%m%d")
    prev_date = datetime.strptime(str(prev_tourney_date), "%Y%m%d")
    time_diff = (upcoming_date - prev_date).days / 365.25
    return prev_age + time_diff

# Add age column and clean output
scraped_future_matches['age'] = scraped_future_matches.apply(lambda x: calculateAge(x.tourney_date, x.prev_tourney_date, x.prev_age), axis=1)
scraped_future_matches = scraped_future_matches.drop(columns = ['prev_age', 'prev_tourney_date'])

# Dataframe with info about all distinct tournaments
unique_matches = full_df.drop_duplicates(subset=['tourney_name'], keep='last')[['tourney_id', 'tourney_name', 'grand_slam', 'surface', 'best_of']]
scraped_future_matches = scraped_future_matches.merge(unique_matches, on='tourney_name')

# Concatenate historical data and future data
all_df = pd.concat([full_df, scraped_future_matches]).drop(columns=['index', 'Unnamed: 0']).reset_index()
all_df.tail()

,index,tourney_id,tourney_name,surface,tourney_date,id,seed,name,hand,ht,...,opp_bpSaved,opp_bpFaced,rank,rank_points,opp_rank,opp_rank_points,fantasy_pts,opp_fantasy_pts,grand_slam,prizepicks_line
100200,100200,2021-9210,Laver Cup,Hard,20210924,106401,NaN,Nick Kyrgios,R,193.0,...,NaN,NaN,95.0,798.0,3.0,8350.0,NaN,NaN,0,NaN
100201,100201,2021-9210,Laver Cup,Hard,20210924,124187,NaN,Reilly Opelka,R,211.0,...,NaN,NaN,19.0,2341.0,10.0,3440.0,NaN,NaN,0,NaN
100202,0,2021-540,Wimbledon,Grass,20220806,104745,NaN,Rafael Nadal,L,185.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,23.0
100203,1,2021-540,Wimbledon,Grass,20220806,104925,NaN,Novak Djokovic,R,188.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,11.0
100204,2,2021-540,Wimbledon,Grass,20220806,106401,NaN,Nick Kyrgios,R,193.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,16.0


## Calculate Historical Averages

Calculates historical averages for match statistics by using a rolling average. Since predictions for `fantasy_pts` will be made before the match, in-game statistics like aces, double faults, break points, etc. are used by finding a player's average for that statistic up until the match.


*Possible future development may include changing the rolling average to a window (i.e. average of stats over the last year instead of a player's whole career)

In [14]:
# Keep using full_df as name for consistency
full_df = all_df

# Create new columns for the averages
cols = ['avg_fantasy_pts', 'avg_ace', 'avg_df', 'avg_svpt', 'avg_FirstIn', 'avg_FirstWon', 'avg_SecondWon', 'avg_SvGms', 'avg_bpSaved', 'avg_bpFaced']
for col in cols:
    full_df[col] = None

# For every unique person, see a dataframe of just their games (temp_df)
for i in range(len(pd.unique(full_df['id']))):
    temp_player = pd.unique(full_df['id'])[i].copy()
    temp_df = full_df[full_df['id'] == temp_player]
    
    # For every game in this dataframe, create a list for the averages  
    avg_fantasy_pts = []
    avg_ace = []
    avg_df = []
    avg_svpt = []
    avg_FirstIn = []
    avg_FirstWon = []
    avg_SecondWon = []
    avg_SvGms = []
    avg_bpSaved = []
    avg_bpFaced = []
    
    # Calculate these averages
    for j in range(len(temp_df)):
        avg_fantasy_pts.append(np.mean(temp_df['fantasy_pts'][0:j]))
        avg_ace.append(np.mean(temp_df['ace'][0:j]))
        avg_df.append(np.mean(temp_df['df'][0:j]))
        avg_svpt.append(np.mean(temp_df['svpt'][0:j]))
        avg_FirstIn.append(np.mean(temp_df['FirstIn'][0:j]))
        avg_FirstWon.append(np.mean(temp_df['FirstWon'][0:j]))
        avg_SecondWon.append(np.mean(temp_df['SecondWon'][0:j]))
        avg_SvGms.append(np.mean(temp_df['SvGms'][0:j]))
        avg_bpSaved.append(np.mean(temp_df['bpSaved'][0:j]))
        avg_bpFaced.append(np.mean(temp_df['bpFaced'][0:j]))
        
    # Append these averages to the unfiltered dataframe
    full_df.loc[full_df['id'] == temp_player, 'avg_fantasy_pts'] = avg_fantasy_pts
    full_df.loc[full_df['id'] == temp_player, 'avg_ace'] = avg_ace
    full_df.loc[full_df['id'] == temp_player, 'avg_df'] = avg_df
    full_df.loc[full_df['id'] == temp_player, 'avg_svpt'] = avg_svpt
    full_df.loc[full_df['id'] == temp_player, 'avg_FirstIn'] = avg_FirstIn
    full_df.loc[full_df['id'] == temp_player, 'avg_FirstWon'] = avg_FirstWon
    full_df.loc[full_df['id'] == temp_player, 'avg_SecondWon'] = avg_SecondWon
    full_df.loc[full_df['id'] == temp_player, 'avg_SvGms'] = avg_SvGms
    full_df.loc[full_df['id'] == temp_player, 'avg_bpSaved'] = avg_bpSaved
    full_df.loc[full_df['id'] == temp_player, 'avg_bpFaced'] = avg_bpFaced

If the player has faced a given opponent before, calculate the average `fantasy_pts` scored in those matches.

*This can take a very long time to run. Future development will include optimizing these for loops to reduce complexity (if possible)

In [15]:
# Column for H2H fantasy pts
full_df['h2h_fantasy_pts'] = None

# For every unique player-opponent combination, see a dataframe of just their games (temp_df)
for i in range(len(full_df.loc[:, ['id', 'opp_id']].drop_duplicates().values)):
    temp_player = full_df.loc[:, ['id', 'opp_id']].drop_duplicates().values[i][0]
    temp_opponent = full_df.loc[:, ['id', 'opp_id']].drop_duplicates().values[i][1]
    temp_df = full_df[(full_df['id'] == temp_player) & (full_df['opp_id'] == temp_opponent)]

    # For every game in this dataframe, create a list for the averages  
    h2h_fantasy_pts = []
    
    for j in range(len(temp_df)):
        h2h_fantasy_pts.append(np.mean(temp_df['fantasy_pts'][0:j]))
        
    # Append these averages to the unfiltered dataframe
    full_df.loc[(full_df['id'] == temp_player) & (full_df['opp_id'] == temp_opponent), 'h2h_fantasy_pts'] = h2h_fantasy_pts

## Fill Missing Data

For columns with missing data, fill these values (either with a specific value or an average) depending on the column. 

In [16]:
# Initialize dictionary with column, what to fill null values with
metadata_cols_to_fill = {
    'seed': 50,
    'opp_seed': 50,
    'rank_points': 0,
    'opp_rank_points': 0,
    'ht': full_df['ht'].mean(),
    'opp_ht': full_df['opp_ht'].mean(),
    'age': full_df['age'].mean(),
    'opp_age': full_df['opp_age'].mean(),
    'hand': 'R',
    'opp_hand': 'R'
}
# Fills metadata columns above
for col in metadata_cols_to_fill:
    full_df.loc[full_df[col].isna(), col] = metadata_cols_to_fill.get(col)     
    
# Initialize dictionary with column, column to use to impute nulls with average
stat_cols_to_fill = {
    'avg_fantasy_pts': 'fantasy_pts',
    'avg_ace': 'ace',
    'avg_df': 'df',
    'avg_svpt': 'svpt',
    'avg_FirstIn': 'FirstIn',
    'avg_FirstWon': 'FirstWon',
    'avg_SecondWon': 'SecondWon',
    'avg_SvGms': 'SvGms',
    'avg_bpSaved': 'bpSaved',
    'avg_bpFaced': 'bpFaced',
    'h2h_fantasy_pts': 'avg_fantasy_pts'
}
# Function to fill stat columns above
for col in stat_cols_to_fill:
    full_df.loc[full_df[col].isna(), col] = full_df[full_df[col].isna()][stat_cols_to_fill.get(col)].median()

## Cleaning Model Dataframe

Reduce dimensionality in the cleaned dataframe where possible, and drop remaining null values in the `fantasy_pts` column.

*There were some columns necessary to calculate `fantasy_pts` (such as `ace`) that included nulls. This resulted in the fantasy_pts column having a small number of nulls. It is best to drop these values for now since imputing averages for the response variable may cause our model to derivive incorrect conclusions about the data.

In [52]:
# Reducing dimensionality by differencing certain columns
full_df['rank_points_diff'] = full_df['rank_points'] - full_df['opp_rank_points']
full_df['seed_diff'] = full_df['seed'] - full_df['opp_seed']

# Select modeling columns
model_df = full_df[[
    'name', 'opp_name', 'tourney_date', # Non-modeling vars, used to track predictions
    'grand_slam', 'best_of', 'surface', # Tourney vars
    'age', 'opp_age', 'seed_diff', 'rank_points_diff', # Player info
    'avg_fantasy_pts', 'avg_ace', 'avg_df', 'avg_svpt', 'avg_FirstIn', # Stat avgs
    'avg_FirstWon', 'avg_SecondWon', 'avg_SvGms', 'avg_bpSaved', 'avg_bpFaced', # Stat avgs
    'h2h_fantasy_pts', # H2H avgs
    'fantasy_pts', 'prizepicks_line' # Response variables
]]

# Drop cols with null for both fantasy_pts and prizepicks_line (i.e. past matches with no fantasy points)
model_df = model_df.dropna(subset=['fantasy_pts', 'prizepicks_line'], how='all')
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92757 entries, 0 to 100204
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              92757 non-null  object 
 1   opp_name          92754 non-null  object 
 2   tourney_date      92757 non-null  int64  
 3   grand_slam        92757 non-null  int64  
 4   best_of           92757 non-null  int64  
 5   surface           92757 non-null  object 
 6   age               92757 non-null  float64
 7   opp_age           92757 non-null  float64
 8   seed_diff         92757 non-null  float64
 9   rank_points_diff  92757 non-null  float64
 10  avg_fantasy_pts   92757 non-null  object 
 11  avg_ace           92757 non-null  object 
 12  avg_df            92757 non-null  object 
 13  avg_svpt          92757 non-null  object 
 14  avg_FirstIn       92757 non-null  object 
 15  avg_FirstWon      92757 non-null  object 
 16  avg_SecondWon     92757 non-null  objec

# MODEL

## Import Modeling Packages

In [26]:
import seaborn as sns
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## Preprocess Data

In [70]:
# Create dummy variables for surface
dummies_df = pd.get_dummies(data=model_df, columns=['surface'])

In [71]:
# Split into matches to fit and matches to predict
matches_to_fit = dummies_df[pd.isnull(dummies_df['prizepicks_line'])]
matches_to_predict = dummies_df[pd.notna(dummies_df['prizepicks_line'])]

In [72]:
# Save numeric variables (and surface)
num_vars = [
    'grand_slam', 'best_of', 'surface_Carpet', 'surface_Clay', 'surface_Grass', 'surface_Hard', # Tourney vars
    'age', 'opp_age', 'seed_diff', 'rank_points_diff', # Player vars
    'avg_fantasy_pts', 'avg_ace', 'avg_df', 'avg_svpt', 'avg_FirstIn', # Stat avgs
    'avg_FirstWon', 'avg_SecondWon', 'avg_SvGms', 'avg_bpSaved', 'avg_bpFaced', # Stat avgs
    'h2h_fantasy_pts', # H2H avgs
    'fantasy_pts'] # Response var
train_df = matches_to_fit[num_vars]
test_df = matches_to_predict[num_vars]

In [73]:
# Train-test split
X_train = train_df.drop(['fantasy_pts'], axis = 1)
y_train = train_df['fantasy_pts']

X_test = test_df.drop(['fantasy_pts'], axis = 1)

## Fit and Predict Random Forest Regressor

It is clear from the `ml_models` notebook that the random forest regressor performs the best (mean-absolute-error of about 3.86), so we will select this model and perform additional analysis.

In [74]:
# Fit a random forest and predict
rfor = RandomForestRegressor(max_depth=19, max_features=1.0)
rfor.fit(X_train, y_train)

# Get model predictions
y_pred = rfor.predict(X_test)

array([7.1665469 , 5.04038215, 9.80851914])

In [86]:
# Return results
results = matches_to_predict[['name', 'opp_name', 'tourney_date', 'prizepicks_line']]
results['prediction'] = y_pred
results.head()

<ipython-input-86-d21da93d955d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['prediction'] = y_pred


,name,opp_name,tourney_date,prizepicks_line,prediction
100202,Rafael Nadal,NaN,20220806,23.0,6.680365
100203,Novak Djokovic,NaN,20220806,11.0,4.827022
100204,Nick Kyrgios,NaN,20220806,16.0,10.419166
